#### 최근접 이웃 분류(k-Nearest Neighbors Classification)
> - k개의 가장 가까운 이웃을 기반으로 데이터 포인트를 분류하는 방법
> - 입력된 새로운 데이터 포인트에 대해 학습 데이터 집합에서 가장 가까운 k개의 이웃을 찾는다.
> - 이 k개의 이웃 중 가장 많은 클래스 레이블을 새로운 데이터 포인트의 예측 클래스로 결정한다.
> - 즉, 새로운 데이터 포인트는 k개의 가장 가까운 이웃이 속한 다수의 클래스로 분류된다.

> - 최근접 이웃 분류 알고리즘은 데이터 분포에 대한 가정이 적고, 비모수적 방법이기 때문에 다양한 분류 문제에 적용할 수 있다.
> - 특히 선형 분리가 어려운 복잡한 데이터 집합에서 좋은 성능을 보일 수 있다.

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

diabetes_data = pd.read_csv('diabetes.csv')
print(diabetes_data['Outcome'].value_counts())
print()

print(diabetes_data.describe())
print("\n")

print('데이터 세트 Null 값 갯수 ',diabetes_data.isnull().sum().sum(), "\n")

diabetes_data.head(3)

Outcome
0    500
1    268
Name: count, dtype: int64

       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.500000   
75%       6.000000  140.250000      80.000000      32.000000  127.250000   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age     Outcome  
count  768.000000                768.000000  768.000000  768.000000  
mean    31.992578                  0.471876   33.240885    0.348958  
std      7.884160                  0.331329   11.760

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [29]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    
    accuracy = accuracy_score(y_test , pred)
    
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    
    f1 = f1_score(y_test,pred)
    
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion, "\n")
    
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc), "\n")


def precision_recall_curve_plot(y_test=None, pred_proba_c1=None):
    
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출. 
    precisions, recalls, thresholds = precision_recall_curve( y_test, pred_proba_c1)
    
    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
    
    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [30]:
zero_features = ['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI']

diabetes_data[zero_features] = diabetes_data[zero_features].replace(0, diabetes_data[zero_features].mean())

diabetes_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.681605,72.254807,26.606479,118.660163,32.450805,0.471876,33.240885,0.348958
std,3.369578,30.436016,12.115932,9.631241,93.080358,6.875374,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.750000,64.000000,20.536458,79.799479,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,79.799479,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [31]:
X = diabetes_data.iloc[:, :-1]
y = diabetes_data.iloc[:, -1]

# StandardScaler 클래스를 이용해 피처 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler( )
X_scaled = scaler.fit_transform(X)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# 그리드 서치 수행
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'metric': ['euclidean', 'manhattan', 'cosine']
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)
knr = grid_search.fit(X_train, y_train)


# 최적의 하이퍼 파라미터 출력
print("Best Parameters:", knr.best_params_)

# 훈련 데이터 성능 평가
print("Train Accuracy:", knr.score(X_train, y_train))

# 테스트 데이터 성능 평가 
print("Test Accuracy:", knr.score(X_test, y_test), "\n\n")


Best Parameters: {'metric': 'manhattan', 'n_neighbors': 7}
Train Accuracy: 0.8241042345276873
Test Accuracy: 0.7272727272727273 




In [35]:
knr = KNeighborsClassifier(n_neighbors=7,  metric='manhattan')

knr.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = knr.predict(X_test)
print(y_pred, "\n")


# 오차행렬
pred_proba = knr.predict_proba(X_test)[:, 1] # 분류 결정 예측 확률 = 각 클래스에 대한 예측 확률을 반환
print(pred_proba[:4])

get_clf_eval(y_test , y_pred, pred_proba)


# 훈련 데이터 성능 평가
print("Train Accuracy:", knr.score(X_train, y_train))

# 테스트 데이터 성능 평가 
print("Test Accuracy:", knr.score(X_test, y_test), "\n\n")

[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1
 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0 0
 0 0 0 1 1 0] 

[0.42857143 0.28571429 1.         0.28571429]
오차 행렬
[[84 18]
 [24 28]] 

정확도: 0.7273, 정밀도: 0.6087, 재현율: 0.5385,    F1: 0.5714, AUC:0.7600 

Train Accuracy: 0.8241042345276873
Test Accuracy: 0.7272727272727273 


